In [18]:
import numpy as np
import torch
from env import MultiAgentEnv
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
import pandas as pd
import json
import torch.nn as nn
import torch.optim as optim
import random
from collections import deque
from new_env import New_env
import gymnasium as gym
from sklearn.metrics import f1_score, accuracy_score
from imblearn.over_sampling import RandomOverSampler

In [19]:
X_train_df = pd.read_csv('/Users/louisedurand-janin/Documents/GitHub/HrFlow_Data_Challenge/data/X_train.csv')
X_train_df['employee embedding'] = X_train_df['employee embedding'].apply(lambda x: np.array(json.loads(x), dtype=np.float32))
X_train_df['company embedding'] = X_train_df['company embedding'].apply(lambda x: np.array(json.loads(x), dtype=np.float32))

In [20]:
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_train_df, y_train)

In [22]:
pd.DataFrame(y_resampled).value_counts()

0
0    17793
1    17793
2    17793
3    17793
Name: count, dtype: int64

In [24]:
X_resampled.head()

,Unnamed: 0,id,employee embedding,company embedding
0,0,0,"[0.013262551, -0.37616727, -0.61382735, 0.3597...","[0.39761704, 0.011816107, 0.1875633, 0.2807579..."
1,1,1,"[-0.28273815, -0.012908442, 0.20536786, 0.4824...","[0.3105131, -0.33802372, 0.023328865, 0.698560..."
2,2,2,"[0.39998972, -0.29847285, -0.21097325, -0.0329...","[0.19704665, 0.062399972, 0.25658086, 0.255205..."
3,3,3,"[0.39998972, -0.29847285, -0.21097325, -0.0329...","[0.6501612, -0.17757246, 0.14481378, 0.2488950..."
4,4,4,"[0.01012421, 0.057610452, -0.1217429, 0.057732...","[0.5966811, -0.061620507, 0.056970306, 0.38894..."


In [3]:
y_train = np.load('data/y_train.npy', allow_pickle=True)

In [25]:
# Convertir les colonnes d'embeddings en tensors PyTorch
employee_embedding_tensor = torch.tensor(np.vstack(X_resampled['employee embedding'].values), dtype=torch.float32)
company_embedding_tensor = torch.tensor(np.vstack(X_resampled['company embedding'].values), dtype=torch.float32)

# Concaténer les deux tensors le long de la dimension appropriée (axis=1 pour ajouter des colonnes)
combined_tensor = torch.cat([employee_embedding_tensor, company_embedding_tensor], dim=1)

In [5]:
def collect_samples(env, horizon, disable_tqdm=False, print_done_states=False):
    s, _ = env.reset(True)
    S = []
    A = []
    rewards = []
    S2 = []
    SA = []

    D = []
    #print("s", s)
    #print("true position", env.y[env.index])

    for _ in tqdm(range(horizon), disable=disable_tqdm):
        a = env.sample_action()
        s2, emb2s2, r, done, _ = env.step(a)
        #print("next state preidcted", s2)
        #print("next embedding state ", emb2s2)
        S.append(s)
        #print('shape S', len(S))
        A.append(a)
        sa = torch.cat([s, torch.tensor(a)])
        SA.append(sa)
        rewards.append(r)
        #print("reward", r)
        S2.append(emb2s2)
        D.append(done)
        if done:
            s, _ = env.reset(False)
            #print("new embedding", s)
            #print("true position", env.y[env.index])
            #print("done!")
        else:
            s = emb2s2
    S2 = np.array(S2)
    S = np.array(S)
    A = np.array(A)
    D = np.array(D)
    SA = np.array(SA)
    rewards = np.array(rewards)
    return S, A, rewards, S2, D, SA

In [6]:
career_env = New_env(combined_tensor, y_train)
S, A, R, S2, D, SA = collect_samples(career_env, 1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

/var/folders/hz/p_k7b1g12kz9ykb1cx0n70w00000gn/T/ipykernel_15963/1229601373.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sa = torch.cat([s, torch.tensor(a)])
100%|██████████| 1000/1000 [00:00<00:00, 3018.33it/s]


In [7]:
SA.shape

(1000, 66)

In [8]:
D.flatten()

array([False, False, False, False, False,  True, False, False, False,
       False, False, False,  True, False, False, False, False,  True,
       False, False, False, False, False, False,  True, False, False,
        True, False, False,  True, False, False, False, False, False,
        True, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False,  True, False, False, False, False,  True,
       False, False, False, False, False,  True, False, False, False,
       False,  True, False, False, False, False,  True, False, False,
       False, False, False, False,  True, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False,  True, False, False, False, False,
        True, False,

In [9]:
def rf_fqi(S, A, R, S2, D, SA, iterations, nb_actions=2, gamma=0.9, disable_tqdm=False):
    nb_samples = S.shape[0]
    Qfunctions = []
    #SA = np.append(S,A,axis=1)
    for iter in tqdm(range(iterations), disable=disable_tqdm):
        #print("iteration", iter)
        if iter==0:
            value=R.copy()
        else:
            Q2 = np.zeros((nb_samples,nb_actions))
            for a2 in range(nb_actions):
                A2 = torch.full((S2.shape[0], 1), a2)
                #print(A2.shape)
                S2A2 = torch.cat([torch.from_numpy(S2), A2], dim=1)
                S2A2_array = np.array(S2A2)
                #print("shaoe A2S2", S2A2_array.shape)
                predictions = Qfunctions[-1].predict(S2A2_array)
                #print("shape pred", predictions.shape)
                Q2[:,a2] = Qfunctions[-1].predict(S2A2_array)
            max_Q2 = np.max(Q2,axis=1)
            value = R + gamma*np.dot((1-D.flatten()),max_Q2)
            #print("value", value.shape)
        Q = RandomForestRegressor()
        Q.fit(SA,value)
        Qfunctions.append(Q)
    return Qfunctions

In [10]:
qfunctions = rf_fqi(S, A, R, S2, D, SA, 5)

100%|██████████| 5/5 [00:09<00:00,  1.94s/it]


In [36]:

def greedy_action(Q,s,nb_actions=2):
    Qsa = []
    for a in range(nb_actions):
        #print("s", s)
        print("a", torch.tensor([a]))
        sa = torch.cat([s, torch.tensor([a])], dim=0)
        sa = np.array(sa)
        #print(sa.reshape(-1,1).shape)
        print("prediction", Q.predict(sa.reshape(1, -1)))
        Qsa.append(Q.predict(sa.reshape(1, -1)))
    print("Qsa", Qsa)
    return np.argmax(Qsa)

In [35]:
combined_tensor

tensor([[ 0.0133, -0.3762, -0.6138,  ..., -0.4219, -0.3206, -0.4043],
        [-0.2827, -0.0129,  0.2054,  ..., -0.1548,  0.0759, -0.6390],
        [ 0.4000, -0.2985, -0.2110,  ..., -0.5598,  0.1031, -0.1492],
        ...,
        [-0.2505, -0.2068,  0.5609,  ..., -0.1784, -0.3287, -0.4991],
        [-0.2505, -0.2068,  0.5609,  ..., -0.8119,  0.0043,  0.2834],
        [-0.6191, -0.0630, -0.2045,  ..., -0.3501, -0.6396, -0.5535]])

In [37]:
pred=[]
y_pred = []
s,_ =  career_env.reset(True)

for t in range(5):
    for k in range(5):
        a = greedy_action(qfunctions[-1],s)
        predicted_next_position, predicted_next_emb_state, reward, d, _ = career_env.step(a)
        print("s", s)
        s = predicted_next_emb_state

        if d:
            break
    pred.append(s)
    y_pred.append(predicted_next_position.item())
    s,_=career_env.reset(False)


a tensor([0])
prediction [-3.73505772e+11]
a tensor([1])
prediction [-3.73505772e+11]
Qsa [array([-3.73505772e+11]), array([-3.73505772e+11])]
s tensor([ 0.0133, -0.3762, -0.6138,  0.3597,  0.1939,  0.3880, -0.0020, -0.0196,
         0.1618,  0.0734, -0.6540,  0.4437,  0.8655, -0.4937,  0.6503, -0.6351,
        -0.5592, -0.7621,  0.9693, -0.4775, -0.4163,  0.8326,  0.4377,  0.0817,
         0.6264,  0.4705, -0.2223,  0.6112, -0.8858, -0.1964, -0.4155, -0.8893,
         0.3976,  0.0118,  0.1876,  0.2808, -0.3438, -0.2601, -0.2431, -0.0504,
         0.1592, -0.1317, -0.5096, -0.2355,  0.8284, -0.6994,  0.4634,  0.3996,
        -0.7340, -0.4339,  0.7385, -0.2088, -0.0800,  0.3684,  0.4964, -0.2953,
         0.2941,  0.5663, -0.4504,  0.6678, -0.9076, -0.4219, -0.3206, -0.4043,
         0.0000])
a tensor([0])
prediction [-3.73505772e+11]
a tensor([1])
prediction [-3.73505772e+11]
Qsa [array([-3.73505772e+11]), array([-3.73505772e+11])]
s tensor([ 0.0133, -0.3762, -0.6138,  0.3597,  0.1939,

In [26]:
print(y_pred)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [ ]:


class DQNAgent(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQNAgent, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0   # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001

        # Neural Net for Deep-Q learning Model
        self.fc1 = nn.Linear(state_size, 24)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(24, 24)
        self.fc3 = nn.Linear(24, action_size)
        
        self.optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)

    def forward(self, state):
        x = self.relu(self.fc1(state))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        state = torch.FloatTensor(state).unsqueeze(0)
        act_values = self.forward(state)
        return np.argmax(act_values.cpu().data.numpy())

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            state = torch.FloatTensor(state).unsqueeze(0)
            next_state = torch.FloatTensor(next_state).unsqueeze(0)
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.forward(next_state).cpu().data.numpy())
            target_f = self.forward(state)
            target_f[0][action] = target
            self.optimizer.zero_grad()
            loss = nn.MSELoss()(target_f, target_f)  # Note: You might want to adjust this loss calculation.
            loss.backward()
            self.optimizer.step()

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


In [21]:
def greedy_action(network, state):
    device = "cuda" if next(network.parameters()).is_cuda else "cpu"
    with torch.no_grad():
        Q = network(torch.Tensor(state).unsqueeze(0).to(device))
        return torch.argmax(Q).item()

In [ ]:
import numpy as np
import torch
import torch.nn as nn
#from solutions.replay_buffer2 import ReplayBuffer
#from solutions.dqn_greedy_action import greedy_action

class dqn_agent:
    def __init__(self, config, model):
        device = "cuda" if next(model.parameters()).is_cuda else "cpu"
        self.gamma = config['gamma']
        self.batch_size = config['batch_size']
        self.nb_actions = config['nb_actions']
        #self.memory = ReplayBuffer(config['buffer_size'], device)
        self.epsilon_max = config['epsilon_max']
        self.epsilon_min = config['epsilon_min']
        self.epsilon_stop = config['epsilon_decay_period']
        self.epsilon_delay = config['epsilon_delay_decay']
        self.epsilon_step = (self.epsilon_max-self.epsilon_min)/self.epsilon_stop
        self.model = model 
        self.criterion = torch.nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=config['learning_rate'])
    
    def gradient_step(self):
        if len(self.memory) > self.batch_size:
            X, A, R, Y, D = self.memory.sample(self.batch_size)
            QYmax = self.model(Y).max(1)[0].detach()
            #update = torch.addcmul(R, self.gamma, 1-D, QYmax)
            update = torch.addcmul(R, 1-D, QYmax, value=self.gamma)
            QXA = self.model(X).gather(1, A.to(torch.long).unsqueeze(1))
            loss = self.criterion(QXA, update.unsqueeze(1))
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step() 
    def train(self, env, max_episode):
        episode_return = []
        episode = 0
        episode_cum_reward = 0
        state, _ = env.reset()
        epsilon = self.epsilon_max
        step = 0

        while episode < max_episode:
            # update epsilon
            if step > self.epsilon_delay:
                epsilon = max(self.epsilon_min, epsilon-self.epsilon_step)

            # select epsilon-greedy action
            if np.random.rand() < epsilon:
                action = env.action_space.sample()
            else:
                action = greedy_action(self.model, state)

            # step
            next_state, reward, done, trunc, _ = env.step(action)
            self.memory.append(state, action, reward, next_state, done)
            episode_cum_reward += reward

            # train
            self.gradient_step()

            # next transition
            step += 1

            if done:
                episode += 1
                print("Episode ", '{:3d}'.format(episode), 
                      ", epsilon ", '{:6.2f}'.format(epsilon), 
                      ", batch size ", '{:5d}'.format(len(self.memory)), 
                      ", episode return ", '{:4.1f}'.format(episode_cum_reward),
                      sep='')
                state, _ = env.reset()
                episode_return.append(episode_cum_reward)
                episode_cum_reward = 0
            else:
                state = next_state

        return episode_return

